In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def split_data_by_bands(filename):
    bands = []
    current_band_data = []

    with open(filename, 'r') as file:
        next(file)

        for line in file:
            if line.strip():  # Check if the line is not empty
                data = line.split()
                current_band_data.append([float(val) for val in data])
            else:
                bands.append(current_band_data)
                current_band_data = []

        if current_band_data:
            bands.append(current_band_data)

    return bands

def filter_by_k_value(df, column_name, value):
    """
    Filter the DataFrame `df` for rows where `column_name` matches `value`.

    Parameters:
        df (pd.DataFrame): The DataFrame to filter (e.g., cb1 or cb2).
        column_name (str): The name of the column to filter by ('kx' or 'ky').
        value (float): The value to filter for in the specified column.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows where the column value matches the input.
    """
    if column_name not in ['kx', 'ky']:
        raise ValueError("Column name must be 'kx' or 'ky'")

    filtered_df = df[df[column_name] == value]
    return filtered_df


In [2]:
root = "non_standard_cell_pbc/length_10nm/3dbands/potential_0"

In [11]:
data = split_data_by_bands(filename=f"{root}/bulkek.dat")
data = np.array(data)

In [12]:
data.shape

(96, 4141, 5)

In [93]:
cb1 = data[48][:, [0, 1, 2, 4 ]]
cb2 = data[49][:, [0, 1, 2, 4 ]]

In [144]:
from copy import deepcopy

cb1t = deepcopy(cb1)

In [145]:
kpts = cb1t[:, [0, 1, 2]]

In [146]:
kpt = np.array([1, 0, 0])
# rotation by 30 should yield [0.5, 0.87, 0]

theta = np.deg2rad(30)
M = np.array(
    [
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1],
    ],
)
M
M.dot(kpt.T).T

array([0.8660254, 0.5      , 0.       ])

In [147]:
cb1t[:, [0, 1, 2]] = M.dot(kpts.T).T

In [148]:
dirac = np.array([0.5,0.33,0])
dirac_t = M.dot(dirac)
dirac_t
cb1t[:, [0, 1, 2]] -= dirac_t

In [152]:
cb1t[cb1t[:, :3] == [0, 0, 0]]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 4 but corresponding boolean dimension is 3

In [158]:
np.where(cb1t[:, 0] == 0)

(array([1716]),)

In [159]:
K_ind = 1716

array([0.        , 0.        , 0.        , 0.18483814])